In [1]:
import sys
sys.path.append('../')
import functions
exec(open('./functions.py').read())


C:\Anaconda\envs\uni\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\home\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
... OK (Got response in 0.49 seconds)
Writing emoji data to C:\Users\home\.demoji\codes.json ...
... OK
[nltk_data] Downloading package stopwords to
[nltk_data]     

In [2]:
#load labelled data (generated in file 2.preprocessing)
data = pickle.load(open('data_n_models/eng_df_labelled.pkl', "rb"))
data.head()

,index,user_id,user_statuses,name,tweet,retweets,location,created,followers,is_user_verified,favorite_count,in_reply_to_status_id,keyword,language,clean,clean_vader,compound,lem,lemma
0,0,819101311715131392,77,Ali09685762,@Zalando give me my money you owe me!!!,0,,07-Mar-2021,6,False,0.0,NaN,@zalando,en,money owe,give money you owe me!!!,0.0000,"(money, owe)",money owe
1,1,1189512849472643072,105,Labellerr1,#VirtualStyling and try-on increases the conve...,0,chandigarh,06-Mar-2021,18,False,0.0,NaN,@zalando,en,virtualstyling try increases conversion rate l...,VirtualStyling and try-on increases the conver...,0.7650,"(virtualstyling, try, increases, conversion, r...",virtualstyle try increase conversion rate larg...
2,2,2837691996,270,puneetjindalisb,#VirtualStyling and try-on increases the conve...,0,"Chandigarh, India",06-Mar-2021,149,False,0.0,NaN,@zalando,en,virtualstyling try increases conversion rate l...,VirtualStyling and try-on increases the conver...,0.7650,"(virtualstyling, try, increases, conversion, r...",virtualstyle try increase conversion rate larg...
3,3,1125728513666048000,9604,DommeLineCoUk,@missbellalugosi @Zalando Problem starts at Ad...,0,,06-Mar-2021,2193,False,1.0,1.368196e+18,@zalando,en,problem starts adidas selling latex websites,Problem starts Adidas. They're selling latex t...,-0.4019,"(problem, starts, adidas, selling, latex, webs...",problem start adida sell latex website
4,4,2614256724,35470,medboyUK,@missbellalugosi @Zalando Welcome to my world ...,0,"West Midlands, UK",06-Mar-2021,2503,False,1.0,1.368196e+18,@zalando,en,welcome world sneering disdain,Welcome world sneering and disdain!,-0.1007,"(welcome, world, sneering, disdain)",welcome world sneer disdain


#Labelling bot related tweets
After the observations of the dataset, it was noticed that there are accounts that offer special scripts to buy goods on zalando as well as tweets by users of such scripts. Since this data is not related to the goal of our project we decided to filter them out. 

In order to identify those tweets 2 steps are performed:

In [42]:
def classify_bots(df):
    #collect all the account names related to bots and scripts
    names = []
    pat = r'(bot\s|script|bot_)'
    for name in data.name.unique():
        match = re.findall(pat, name, re.IGNORECASE) 
        if len(match) > 0:
            names.append(name)
    #look for a key words to identify a tweet related to bots and scripts
    pattern = r"(script|bot\s|bots\s|bot_|cook|cop^e)"
    df['bot'] = False
    for i, row in df.iterrows():
        match = re.findall(pattern, row.tweet, re.IGNORECASE) 
        if len(match) > 0 or row.isin(names)['name'] == True:
            df['bot'].loc[i] = True
    return df

In [43]:
#apply the function
bot_df = classify_bots(data)

C:\Anaconda\envs\uni\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [44]:
bot_df.bot[61556]

True

In [46]:
#testing results by checking random 10 tweets
sample = bot_df[bot_df.bot == False].sample(n=10)
for i, row in sample.iterrows():
    print(str(i) + ' ' + sample.name[i] + ' ' + sample.tweet[i])

92787 ameIiagray paint me like one of your easter eggs 🥚🎨 @boohoo https://t.co/148AHUgelm
91671 kayleighmayyy_ Me me me #boohootreats 🥺 https://t.co/yYQKLsJMpA
99258 katierichensx @boohoo Cats never win but he’s the sweetest✨ https://t.co/73F4H0nbef
39443 greenlola123 @boohoo Cali the blue Staffordshire arrived with us yesterday 🐶🐶 settling in nicely with her sausage dog friend Lola 😍😍 https://t.co/oDdhltIR8i
41106 vhbnj5 @makeupbyshaniah YOU IS LYINNNNGGG, im glad my sister stopped me from shopping there. Imagine putting on your new boohoo clothes and you start itching I’d be SICKKK
7587 bethanaw @ASOS @katedmlol for surviving a brain bleed and coma 4 years ago today !! and thriving ever since 💫 #ASOSIWD21
14960 baineym02 Ordered a top from asos and it had literally been 8 days since it was meant to arrive, so messaged customer care today and they said it was lost in the warehouse, cheers babe could’ve told me 8 days ago
7348 Ju5t_Jess @ASOS @xxleanne1986xx #ASOSIWD21 you better tag m

In [47]:
bot_df.keyword.value_counts()

asos        36694
boohoo      24852
zalando      2422
@zalando      296
bohoo         168
Name: keyword, dtype: int64

In [48]:
bot_df.bot.value_counts()

False    61741
True      2691
Name: bot, dtype: int64